## The scope of this code is to run the HMM in the selected ROIs (mPFC, AG, temporal pole, and PMC) and generate two dictionary files:
1. the number of events 
2. the boundaries reflecting a start of a new event 

### We also generate plots that are saved in the figs folder

In [ ]:

import warnings
warnings.filterwarnings('ignore')
import sys 
import os    
import glob
from functools import reduce
import numpy as np
from brainiak.eventseg.event import EventSegment
import nibabel as nib
from scipy.stats import zscore, norm, ttest_rel
# from nilearn.masking import apply_mask
# from nilearn.image import load_img
# from nilearn.image import resample_to_img
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches



In [ ]:
schaeffer_dir = '/home/jovyan/paranoia_project/schaeffer_dict_roi/'
save_dir = '/home/jovyan/paranoia_project/nh-paranoia/schaef_avg_events/'
mask_dir = '/home/jovyan/paranoia_project/selected_ROI/'


In [4]:
#define plotting function    
def plot_tt_similarity_matrix(ax, data_matrix, bounds, n_TRs, title_text):
    
    ax.imshow(np.corrcoef(data_matrix), cmap = 'viridis')
    ax.set_title(title_text)
    ax.set_xlabel('TR')
    ax.set_ylabel('TR')
    
    # plot the boundaries 
    bounds_aug = np.concatenate(([0], bounds, [n_TRs]))
    
    for i in range(len(bounds_aug) - 1):
        rect = patches.Rectangle(
            (bounds_aug[i], bounds_aug[i]),
            bounds_aug[i+1] - bounds_aug[i],
            bounds_aug[i+1] - bounds_aug[i],
            linewidth = 2, edgecolor = 'w',facecolor = 'none'
        )
        ax.add_patch(rect)


In [5]:
label_node = pd.read_csv(mask_dir + '_masks/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.Centroid_RAS.csv')
label_node['ROI Name']


0                  7Networks_LH_Vis_1
1                  7Networks_LH_Vis_2
2                  7Networks_LH_Vis_3
3                  7Networks_LH_Vis_4
4                  7Networks_LH_Vis_5
                   ...               
95    7Networks_RH_Default_PFCdPFCm_1
96    7Networks_RH_Default_PFCdPFCm_2
97    7Networks_RH_Default_PFCdPFCm_3
98     7Networks_RH_Default_pCunPCC_1
99     7Networks_RH_Default_pCunPCC_2
Name: ROI Name, Length: 100, dtype: object

In [8]:
mask_dir = '/home/jovyan/paranoia_project/selected_ROI/'

a = np.load(mask_dir + 'run1_mPFC.npy')

results = {}  # keys are nodes
bounds = {}
bounds_sm = {}


min_here = 5
max_here = 45
k_array = np.arange(min_here, max_here, 10)
test_ll = np.zeros(len(k_array))

run = int(input('What run do you want?'))


for node in ['AG','mPFC','PMC','temporal_pole']:
    print(node)
    test_ll = np.zeros(len(k_array))
    node_loaded = np.load(mask_dir + f'run{run}_{node}.npy')
       
    #temp
    print('first',node_loaded.shape)
    #n_vox, n_ts, n_subs  = node_loaded.shape
    node_loaded = np.moveaxis(node_loaded, 2,0)
    #node_loaded = np.moveaxis(node_loaded, 1,2)
    

    non_zero_list = []
    for it in range(node_loaded.shape[0]):
        non_zero_list.append(node_loaded[it,:,:][~np.all(node_loaded[it,:,:] == 0, axis = 1)])
    node_loaded = np.array(non_zero_list)
    print(node_loaded.shape)
    node_loaded = np.moveaxis(node_loaded, 1,2)
    print(node_loaded.shape)
    
    n_subs, n_ts, n_vox = node_loaded.shape
    print(node_loaded.shape)
    
    
    
    ### training on half the data
    for i, k in enumerate(k_array):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_train = zscore(movie_train,axis=0)
        
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        movie_test = zscore(movie_test,axis=0)
        
        _, test_ll[i] = movie_HMM.find_events(movie_test)
    max_ind = np.argmax(test_ll)

    print('Max is %d events' % k_array[max_ind])
    print('finished with part 1 for node')
    if max_ind < 6: #so that it doesn't go below 6!
        k_small = np.arange((k_array[max_ind]), (k_array[max_ind]) + 5, 1)
    else:
        k_small = np.arange((k_array[max_ind]) - 5, (k_array[max_ind]) + 5, 1)
    test_ll_small = np.zeros(len(k_small))
    for i, k in enumerate(k_small):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_train = zscore(movie_train,axis=0)
        
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        movie_test = zscore(movie_test,axis=0)
        _, test_ll_small[i] = movie_HMM.find_events(movie_test)

    max_ind_fin = np.argmax(test_ll_small)
    print('Max is %d events' % k_small[max_ind_fin])
    test_ll_small[max_ind_fin]

    movie_group = np.mean(node_loaded, axis=0)
    nTRs = movie_group.shape[0]
    movie_dur = nTRs * 1  # Data acquired every 1  seconds; was 1.5 seconds for Baldassano
    

    results[node] = k_small[max_ind_fin]
    print(results[node])
    np.save(save_dir + f'events_per_roi_run{run}_selected_node_HMM_avg_zscore.npy', results)

    
    HMMsm = EventSegment(n_events=k_small[max_ind_fin], split_merge=True)
    HMMsm.fit(movie_group)
    bounds_s = np.where(np.diff(np.argmax(HMMsm.segments_[0], axis=1)))[0]
    bounds_sm[node] = bounds_s
    print(bounds_s)
    np.save(save_dir + f'event_boundaries_run{run}_selected_node_HMM_avg.npy', bounds_sm)

    event_bounds = np.where(np.diff(np.argmax(movie_HMM.segments_[0], axis = 1)))[0]
    nTRs = movie_test.shape[0]
    f, ax = plt.subplots(1,1, figsize = (10,8))
    title_text = '''
    Overlay the HMM-predicted event boundaries
    on top of the TR-TR correlation matrix
    '''
    plot_tt_similarity_matrix(ax, movie_test, event_bounds, nTRs, title_text)
    plt.savefig('/home/jovyan/paranoia_project/nh-paranoia/figs/' + f'{node}_plot.png')
    plt.close()
    

What run do you want? 1


AG
first (4278, 526, 17)
(17, 4230, 526)
(17, 526, 4230)
(17, 526, 4230)
Max is 25 events
finished with part 1 for node
Max is 25 events
25
[  7  21  52  66  92 126 139 155 186 212 228 254 286 302 313 337 373 396
 414 424 457 474 484 499]
mPFC
first (418, 526, 17)
(17, 418, 526)
(17, 526, 418)
(17, 526, 418)
Max is 5 events
finished with part 1 for node
Max is 8 events
8
[ 59 126 238 303 375 425 499]
PMC
first (8870, 526, 17)
(17, 8870, 526)
(17, 526, 8870)
(17, 526, 8870)
Max is 15 events
finished with part 1 for node
Max is 17 events
17
[ 33  82 100 127 160 190 244 285 302 335 374 399 428 448 480 501]
temporal_pole
first (5517, 526, 17)
(17, 4979, 526)
(17, 526, 4979)
(17, 526, 4979)
Max is 15 events
finished with part 1 for node
Max is 15 events
15
[ 10  40  68 127 161 191 229 259 285 305 377 401 427 499]
